# Получим признаковое описание для датасета DEAP

В этом ноутбуке рассмотрен принцип получения вектора признакового описания из датасета, тот же принцип применен к DREAMER, MEEG.

### Разобьем DEAP на трейн и тестовые подвыборки:

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import random
from typing import List
import matplotlib.pyplot as plt

In [2]:
# путь к DEAP
prepocessed_DEAP_path = 'C:\\Users\\User\\Desktop\\Нейромаркетинг\\data\\DEAP\\data_preprocessed_python\\'
deap_lstdir = os.listdir(prepocessed_DEAP_path)
# deap_lstdir

In [3]:
# создаем общий датасет, который после будем разделять

# проходим по всем испытуемым
subject_dicts = []
for subject_path in deap_lstdir:
    path = os.path.join(prepocessed_DEAP_path, subject_path)
    with open(path, 'rb') as file:
        subject = pickle.load(file, encoding='latin1')
        subject_dicts.append(subject)

In [4]:
def quadrant_fill(val_arous_tuple: tuple):
    valence, arousal = val_arous_tuple
    if valence == 'high' and arousal == 'high':
        return 'HAHV'
    elif valence == 'low' and arousal == 'high':
        return 'HALV'
    elif valence == 'low' and arousal == 'low':
        return 'LALV'
    elif valence == 'high' and arousal == 'low':
        return 'LAHV'

def build_deap_dataframe(subject_dicts) -> pd.DataFrame:
    """
    subject_dicts: список словарей (по одному на каждого испытуемого),
                   каждый словарь должен иметь:
                       'data'   -> shape (40, 40, 8064)
                       'labels' -> shape (40, 4)  (valence, arousal, dominance, liking)

    Возвращает: pd.DataFrame со столбцами:
      [subject_id, video_id, data, valence, arousal, dominance, liking]
    """

    all_rows = []

    for subject_id, subject_dict in enumerate(subject_dicts, start=1):
        data_3d = subject_dict['data']    # shape = (40, 40, 8064)
        labels_2d = subject_dict['labels']  # shape = (40, 4)

        n_trials, n_channels, n_points = data_3d.shape
        n_channels = 32 # берем только ЭЭГ каналы
        fs = 128
        seg_length = 10 * fs  # 10 секунд * 128 Гц = 1280

        for trial_id in range(n_trials):
            # Извлекаем (40, 8064)
            trial_signal = data_3d[trial_id, :n_channels, :]
            
            # Извлекаем метки (valence, arousal, dominance, liking) для данного триала
            valence, arousal, dominance, liking = labels_2d[trial_id]

            # Формируем строку будущего DataFrame
            row_dict = {
                'subject_id': subject_id,
                'video_id': trial_id + 1,
                'data': trial_signal,  
                'valence': valence,
                'arousal': arousal,
                'dominance': dominance,
                'liking': liking
            }
            all_rows.append(row_dict)

    # Создаём DataFrame 
    df = pd.DataFrame(all_rows)
    # сделаем задачу классификации
    df['Low/high valence'] = df['valence'].apply(lambda x: 'low' if x <= 5 else 'high')
    df['Low/high arousal'] = df['arousal'].apply(lambda x: 'low' if x <= 5 else 'high')
    df['quadrant'] = df.apply(lambda x: quadrant_fill((x['Low/high valence'], x['Low/high arousal'])), axis=1)
    return df

def deap_train_test_split(deap_df: pd.DataFrame) -> List[pd.DataFrame]:
    random.seed(42)
    subject_idx = random.sample(range(1, 33),6)
    video_idx = random.sample(range(1, 41),8)

    train_df =  deap_df[(~deap_df['video_id'].isin(video_idx)) & (~deap_df['subject_id'].isin(subject_idx))]
    test_df = deap_df[(deap_df['video_id'].isin(video_idx)) & (deap_df['subject_id'].isin(subject_idx))]
    
    test_new_subj_old_vid = deap_df[(~deap_df['video_id'].isin(video_idx) & (deap_df['subject_id'].isin(subject_idx)))]
    test_new_vid_old_subj = deap_df[(~deap_df['subject_id'].isin(subject_idx) & (deap_df['video_id'].isin(video_idx)))]

    return train_df, test_df, test_new_subj_old_vid, test_new_vid_old_subj # len: 832, 48, 1024, 1040


In [5]:
df = build_deap_dataframe(subject_dicts)
df.head()

,subject_id,video_id,data,valence,arousal,dominance,liking,Low/high valence,Low/high arousal,quadrant
0,1,1,"[[0.948231680995192, 1.65333532651348, 3.01372...",7.71,7.60,6.90,7.83,high,high,HAHV
1,1,2,"[[10.260175049914748, 12.795442725569648, 10.4...",8.10,7.31,7.28,8.47,high,high,HAHV
2,1,3,"[[1.0130495576625123, -1.0678322951836536, 3.9...",8.58,7.54,9.00,7.08,high,high,HAHV
3,1,4,"[[-7.658428424515396, -3.2675578443273143, 0.7...",4.94,6.01,6.12,8.06,low,high,HALV
4,1,5,"[[-1.8111079228929805, -4.7838764286765, -0.52...",6.96,3.92,7.19,6.05,high,low,LAHV


In [6]:
train_df, test_df, test_new_subj_old_vid, test_new_vid_old_subj = deap_train_test_split(df)

### Получим признаки

Расчет признакового описания датасета DEAP для обучения RF регрессора.
Расчет проводился предобработанных данных авторами датасета, которые можно скачать на сайте датасета.
Вектор-описание имеет вид: 2 features × 2 waves × 32 channels + 3 features from alpha waves × 14 asymmetry channels. Общая длина 170 элементов.
First Hjorth parameter (H1) and the Wavelet Energy (WP) from the beta and gamma waves, plus the alpha differential asymmetry from the first Hjorth parameter (H1), the Wavelet Energy (WP) and Wavelet Entropy (WE). Признаки взяты из doi: 10.3390/s21103414

In [1]:
import numpy as np
import pandas as pd
import pickle
import pywt
from scipy.signal import butter, filtfilt, firwin, lfilter
import emd
import os
import mne
import matplotlib.pyplot as plt
from mne.preprocessing import ICA, create_eog_epochs

In [2]:
# Read deap subject
def read_deap_subject_npy(deap_path, subject_id):
    subject_path = f's{subject_id:02d}.npy'
    subject_data = np.load(deap_path + subject_path)
    return subject

def read_deap_subject(deap_path, subject_id):
    subject_path = f'/s{subject_id:02d}.dat'
    with open(deap_path + subject_path, 'rb') as file:
        subject = pickle.load(file, encoding='latin1')
    # subject['data'] = subject_data
    return subject

# Hjorth Parameters
def hjorth_parameters(data):
    activity = np.var(data)
    mobility = np.sqrt(np.var(np.diff(data)) / activity)
    complexity = np.sqrt(np.var(np.diff(np.diff(data))) / np.var(np.diff(data))) / mobility
    return activity, mobility, complexity
    
# Spectral Entropy
def spectral_entropy(data):
    psd = np.abs(np.fft.fft(data))**2
    psd /= np.sum(psd)
    entropy_val = -np.sum(psd * np.log2(psd))
    return entropy_val

# Wavelet Energy and Entropy
# может быть, неверно рассчитывается gamma, beta и alpha коэфф; непонятно, почему нумерация не с нуля
def wavelet_features(data, wavelet='db4', level=3):
    coeffs = pywt.wavedec(data, wavelet, level=level)
    gamma_coeffs, beta_coeffs, alpha_coeffs = coeffs[1], coeffs[2], coeffs[3]
    energy = [np.sum(c**2) for c in [alpha_coeffs, beta_coeffs, gamma_coeffs]]
    entropy_vals = [spectral_entropy(c) for c in [alpha_coeffs, beta_coeffs, gamma_coeffs]]
    return energy, entropy_vals
    
# FIR Bandpass Filter
def bandpass_filter(data, lowcut, highcut, fs, numtaps=101):
    nyquist = 0.5 * fs
    taps = firwin(numtaps, [lowcut / nyquist, highcut / nyquist], pass_zero=False)
    filtered_data = lfilter(taps, 1.0, data)
    return filtered_data

# IMF Energy and Entropy using Empirical Mode Decomposition (EMD)
def emd_features(data):
    imfs = emd.sift.sift(data)
    energy = [np.sum(imf**2) for imf in imfs]
    entropy_vals = [spectral_entropy(imf) for imf in imfs]
    return energy[:3][::-1], entropy_vals[:3][::-1]  # Taking the first 3 IMFs

# Calculate Differential and Rational Asymmetry
def calculate_asymmetry(features_left, features_right):
    differential_asymmetry = features_left - features_right
    rational_asymmetry = features_left / features_right
    return differential_asymmetry, rational_asymmetry

In [3]:
def calculate_features(epoch_data, fs=128):
    """
    Полученные фичи выводятся в формате dict = 
    {
    'features': epoch_features_full,
    'differential_asymmetry': differential_asymmetry_features,
    'rational_asymmetry': rational_asymmetry_features
    }
    
    Размерность векторов-описаний:
    Features Shape: {'alpha': (32, 8), 'beta': (32, 8), 'gamma': (32, 8)} 
    Differential Asymmetry Shape: {'alpha': (14, 8), 'beta': (14, 8), 'gamma': (14, 8))}
    Rational Asymmetry Shape: {'alpha': (14, 8), 'beta': (14, 8), 'gamma': (14, 8))}

    Порядок признаков в векторе:
    [activity, mobility, complexity, se, wavelet_energy, wavelet_entropy, imf_energy, imf_entropy]
    
    """
    # Пары электродов для расчета асимметрии (Fp1-Fp2, AF3-AF4, etc.)
    pairs = [(0, 16), (1, 17), (2, 19), (3, 20), (4, 21), (5, 22), (6, 24), (7, 25), (8, 26), (9, 27), (10, 28), (11, 29), (12, 30), (13, 31)]
    bands = {'alpha': (8, 13), 'beta': (13, 30), 'gamma': (30, 50)}

    # Проводим расчет hjort_parameters и spectral_entropy
    # формат вывода epoch_features = {'alpha/beta/gamma': 32*[activity, mobility, complexity, se]}
    epoch_features = {}
    for band_name, (lowcut, highcut) in bands.items():
            band_features = []
            for channel_data in epoch_data:
                filtered_data = bandpass_filter(channel_data, lowcut, highcut, 128)
                activity, mobility, complexity = hjorth_parameters(filtered_data)
                se = spectral_entropy(filtered_data)
                
    
                band_features.append([activity, mobility, complexity, se])
            epoch_features[band_name] = np.array(band_features)
    
    # Проводим расчет wavelet_features и emd_features
    # формат вывода feat_dic = {'alpha/beta/gamma': 32*[wavelet_energy, wavelet_entropy, imf_energy, imf_entropy]}
    feat_dic = {band : [] for band in bands} 
    full_lst = []
    for channel_data in epoch_data:
        w_features = wavelet_features(channel_data)
        emd_feat = emd_features(channel_data)
        full_lst += [list(w_features) + list(emd_feat)]
    
    for channel in full_lst:
        feature_by_bands = np.array(channel).T
    
        for idx, band  in list(enumerate(bands.keys())):
            feat_dic[band] += [feature_by_bands[idx]]

    # объединяем в 8 компонентный вектор-описания для каждого бэнда
    epoch_features_full = {band: [] for band in bands.keys()}
    for band_name in bands:
        for channel_id, w_features in list(enumerate(feat_dic[band_name])):
            epoch_features_full[band_name] += [list(np.append(epoch_features[band_name][channel_id], w_features))]
    
        epoch_features_full[band_name] = np.array(epoch_features_full[band_name])

    # Проводим расчет asymmetry features для каждого бэнда
    differential_asymmetry_features = {}
    rational_asymmetry_features = {}
    
    for band_name in bands.keys():
        band_features = epoch_features_full[band_name]
        trial_diff_asymmetry = []
        trial_rat_asymmetry = []
        for left, right in pairs:
            diff_asymmetry, rat_asymmetry = calculate_asymmetry(band_features[left], band_features[right])
            trial_diff_asymmetry.append(diff_asymmetry)
            trial_rat_asymmetry.append(rat_asymmetry)
        differential_asymmetry_features[band_name] = np.array(trial_diff_asymmetry)
        rational_asymmetry_features[band_name] = np.array(trial_rat_asymmetry)

    return {
    'features': epoch_features_full,
    'differential_asymmetry': differential_asymmetry_features,
    'rational_asymmetry': rational_asymmetry_features
    }

In [4]:
def combine_features(features_dict):
    """
    Порядок признаков в векторе:
    [activity, mobility, complexity, se, wavelet_energy, wavelet_entropy, imf_energy, imf_entropy]
    """
    combined_features = []
    
    # Combine channel features, excluding the spectral entropy (index 3)
    for band_name, band_features in features_dict['features'].items():
        if band_name in ['beta', 'gamma']: # берем характеристики из бета и гамма бенда
            for channel_features in band_features:
                combined_features.extend(np.delete(channel_features, [1, 2, 3, 5, 6, 7]))  # Оставляем H1 и wavelet_energy
                
    # Combine asymmetry features (only for alpha band)
    for channel_features in features_dict['differential_asymmetry']['alpha']:
        combined_features.extend(np.delete(channel_features, [1, 2, 3, 6, 7])) # Оставляем только H1, wavelet_energy, wavelet_entropy
    

    return np.array(combined_features)

In [10]:
def process_epoch(epoch_data):
    features = calculate_features(epoch_data)
    return combine_features(features)

In [11]:
import swifter

In [17]:
# получим фичи для каждой строки - последних 10 секунд записи
df['features'] = df['data'].swifter.apply(process_epoch)
df

Pandas Apply:   0%|          | 0/832 [00:00<?, ?it/s]

C:\Users\User\AppData\Local\Temp\ipykernel_23424\1715275849.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['features'] = train_df['data'].swifter.apply(process_epoch)


,subject_id,video_id,data,valence,arousal,dominance,liking,Low/high valence,Low/high arousal,quadrant,features
40,2,1,"[[-11.11631040230457, 71.81244100609055, 122.8...",9.00,5.03,7.13,6.62,high,high,HAHV,"[38.67276461061695, 342905.69732023706, 115.15..."
43,2,4,"[[-171.79711783177316, -117.17857114425763, -4...",6.05,1.00,5.04,7.03,high,low,LAHV,"[28.141656438132507, 224581.65034030465, 120.9..."
44,2,5,"[[1.4713609744647984, -38.936057294245664, -66...",5.04,3.00,3.65,5.04,high,low,LAHV,"[39.337478092624025, 327354.6394689991, 100.43..."
47,2,8,"[[15.944453976224564, 11.812813111443923, 7.97...",9.00,9.00,9.00,9.00,high,high,HAHV,"[26.452500488387575, 237332.6259904147, 108.97..."
49,2,10,"[[178.77559036713717, 144.71389583087222, 97.0...",4.99,1.00,9.00,1.00,low,low,LALV,"[25.05151931067869, 217587.3519421451, 105.343..."
...,...,...,...,...,...,...,...,...,...,...,...
1233,31,34,"[[20.488056516461008, 27.592511136336057, 25.5...",1.95,8.03,2.08,1.00,low,high,HALV,"[56.46023892847568, 432153.70573326247, 28.869..."
1235,31,36,"[[0.5274022253354644, -2.4252270719619826, 5.0...",4.97,6.95,1.96,1.96,low,high,HALV,"[40.117415122613686, 309650.8466413178, 21.782..."
1237,31,38,"[[-10.197774986718422, -4.8967558966950655, 4....",1.00,9.00,1.00,1.00,low,high,HALV,"[38.48108050239333, 302274.2442307396, 20.9690..."
1238,31,39,"[[-14.439967892369696, -5.040654886024263, 4.0...",1.00,9.00,1.00,1.01,low,high,HALV,"[35.35617998994652, 283058.9243989627, 18.1022..."
